# Diplodatos Kaggle Competition

This competition is part of the practice work for the subject called 'Aprendizaje Supervisado' of the edition 2020  Diplodatos : https://www.kaggle.com/c/supervisado2020/overview. 
This is a simplified version of the competition for recruiting done by Walmart in the plataform Kaggle (Walmart Recruiting: Trip Type Classification:https://www.kaggle.com/c/walmart-recruiting-trip-type-classification/overview).
The goal is try to predict the trip types of purchases into the supermarket made by costomers. These types are codified in the original dataset.


The main goals are:

1. Learn
1. Try different models and see which one fits the best the given data
1. Get a higher score than the given one in the current baseline example
1. Try to get the highest score in the class :)

In [1]:
# Import the required packages
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

Create a function to load the datasets.

**We present two models**

*First one*
1. We used one hot encoding for Weekday and DepartmentDescription.  We didn't add an aditional column for the NaN values. All transformations are applied to the training and testing datasets...
1. We dropped NaN values, only in the training dataset, and we could see that it's better for our model. 
1. We didn't drop the Upc and FinelineNumber to simplify the process, and we didn't use one hot encoding for that columns, the model went better when we just used those columns as a number. Median value of Upc per department category is used.

In [2]:
def transform_data(train_data_fname, test_data_fname):
    df_train = pd.read_csv(train_data_fname)
    df_train['is_train_set'] = 1
    df_test = pd.read_csv(test_data_fname)
    df_test['is_train_set'] = 0

    
    # we  get the TripType for the train set. To do that, we group by VisitNumber and
    # then we get the max (or min or avg)
    df_train=df_train.dropna(subset=['Upc'])
    y = df_train.groupby(["VisitNumber", "Weekday"], as_index=False).max().TripType
    

    # we remove the TripType now, and concat training and testing data
    # the concat is done so that we have the same columns for both datasets
    # after one-hot encoding
    df_train = df_train.drop("TripType", axis=1)
    df = pd.concat([df_train, df_test])

    # one-hot encoding for the DepartmentDescription    
    df = pd.get_dummies(df, columns=["DepartmentDescription"], dummy_na=False)
   

    # now we add the groupby values
    df = df.groupby(["VisitNumber", "Weekday"], as_index=False).sum()
    
    # finally, we do one-hot encoding for the Weekday
    df = pd.get_dummies(df, columns=["Weekday"], dummy_na=False)
    
    
    # get train and test back
    df_train = df[df.is_train_set != 0]
    df_test = df[df.is_train_set == 0]
    
    X = df_train.drop(["is_train_set"], axis=1)
    yy = None
    XX = df_test.drop(["is_train_set"], axis=1)

    return X, y, XX, yy

Load the data...

In [3]:
X, y, XX, yy = transform_data("../data/train.csv", "../data/test.csv")

In [4]:
X.shape

(66077, 79)

In [5]:
X.head()

,VisitNumber,Upc,ScanCount,FinelineNumber,DepartmentDescription_1-HR PHOTO,DepartmentDescription_ACCESSORIES,DepartmentDescription_AUTOMOTIVE,DepartmentDescription_BAKERY,DepartmentDescription_BATH AND SHOWER,DepartmentDescription_BEAUTY,...,DepartmentDescription_SWIMWEAR/OUTERWEAR,DepartmentDescription_TOYS,DepartmentDescription_WIRELESS,Weekday_Friday,Weekday_Monday,Weekday_Saturday,Weekday_Sunday,Weekday_Thursday,Weekday_Tuesday,Weekday_Wednesday
0,5,6.811315e+10,-1,1000.0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,9,1.070088e+09,3,5719.0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,10,1.700927e+10,3,10073.0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
5,11,2.273895e+10,4,7499.0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
6,12,1.832173e+11,7,15639.0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [6]:
y.shape

(66077,)

In [7]:
y.head()

0    999
1      8
2      8
3     35
4     41
Name: TripType, dtype: int64

In [8]:
XX.shape

(28645, 79)

In [9]:
XX.head()

,VisitNumber,Upc,ScanCount,FinelineNumber,DepartmentDescription_1-HR PHOTO,DepartmentDescription_ACCESSORIES,DepartmentDescription_AUTOMOTIVE,DepartmentDescription_BAKERY,DepartmentDescription_BATH AND SHOWER,DepartmentDescription_BEAUTY,...,DepartmentDescription_SWIMWEAR/OUTERWEAR,DepartmentDescription_TOYS,DepartmentDescription_WIRELESS,Weekday_Friday,Weekday_Monday,Weekday_Saturday,Weekday_Sunday,Weekday_Thursday,Weekday_Tuesday,Weekday_Wednesday
1,7,6.794963e+10,2,13435.0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,8,4.259239e+11,28,58669.0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
7,15,1.019672e+12,9,7880.0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
9,19,1.733264e+12,9,38013.0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
11,23,6.922496e+11,2,8506.0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [10]:
XX.columns

Index(['VisitNumber', 'Upc', 'ScanCount', 'FinelineNumber',
       'DepartmentDescription_1-HR PHOTO', 'DepartmentDescription_ACCESSORIES',
       'DepartmentDescription_AUTOMOTIVE', 'DepartmentDescription_BAKERY',
       'DepartmentDescription_BATH AND SHOWER', 'DepartmentDescription_BEAUTY',
       'DepartmentDescription_BEDDING',
       'DepartmentDescription_BOOKS AND MAGAZINES',
       'DepartmentDescription_BOYS WEAR',
       'DepartmentDescription_BRAS & SHAPEWEAR',
       'DepartmentDescription_CAMERAS AND SUPPLIES',
       'DepartmentDescription_CANDY, TOBACCO, COOKIES',
       'DepartmentDescription_CELEBRATION', 'DepartmentDescription_COMM BREAD',
       'DepartmentDescription_CONCEPT STORES',
       'DepartmentDescription_COOK AND DINE', 'DepartmentDescription_DAIRY',
       'DepartmentDescription_DSD GROCERY',
       'DepartmentDescription_ELECTRONICS',
       'DepartmentDescription_FABRICS AND CRAFTS',
       'DepartmentDescription_FINANCIAL SERVICES',
       'DepartmentD

In [11]:
import xgboost

In [12]:
data_dmatrix = xgboost.DMatrix(data=X,label=y)

Create the model and evaluate it

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
from xgboost import XGBClassifier

In [15]:
XGBClassifier??

Init signature: XGBClassifier(objective='binary:logistic', **kwargs)
Docstring:     
Implementation of the scikit-learn API for XGBoost classification.


Parameters
----------

    n_estimators : int
        Number of boosting rounds.

    max_depth : int
        Maximum tree depth for base learners.
    learning_rate : float
        Boosting learning rate (xgb's "eta")
    verbosity : int
        The degree of verbosity. Valid values are 0 (silent) - 3 (debug).
    objective : string or callable
        Specify the learning task and the corresponding learning objective or
        a custom objective function to be used (see note below).
    booster: string
        Specify which booster to use: gbtree, gblinear or dart.
    tree_method: string
        Specify which tree method to use.  Default to auto.  If this parameter
        is set to default, XGBoost will choose the most conservative option
        available.  It's recommended to study this option from parameters
        document.


In [16]:
my_model = XGBClassifier(objective='binary:logistic', n_estimators=1000, learning_rate=0.1, subsample=0.5)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=1000, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=0.5,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [17]:
from sklearn.metrics import mean_absolute_error

predictions = my_model.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

Mean Absolute Error: 16.726089588377725


In [18]:
from sklearn.metrics import accuracy_score

In [19]:
print("Accuracy: " + str(accuracy_score(y_valid,predictions)))

Accuracy: 0.711410411622276


**And finally**, we predict the unknown label for the testing set

In [20]:
X.shape, XX.shape

((66077, 79), (28645, 79))

In [21]:
# results dataframe is used to store the computed results
results = pd.DataFrame(columns=('clf', 'best_acc'))

In [22]:
results = results.append({'clf': predictions.reshape(-1,1), 'best_acc': accuracy_score(y_valid,predictions)}, ignore_index=True)

In [23]:
results

,clf,best_acc
0,"[[7], [20], [42], [7], [25], [4], [38], [9], [...",0.71141


In [24]:
yy = my_model.predict(XX)

The last thing we do is generating a file that should be *submitted* on kaggle

In [25]:
submission = pd.DataFrame(list(zip(XX.VisitNumber, yy)), columns=["VisitNumber", "TripType"])

In [26]:
submission

,VisitNumber,TripType
0,7,9
1,8,40
2,15,21
3,19,25
4,23,9
...,...,...
28640,191331,9
28641,191335,32
28642,191342,8
28643,191345,39


In [27]:
submission.to_csv("../data/submission.csv", header=True, index=False)

Create a function to load the datasets.

**Second approach**

1. We used one hot encoding for Weekday.  DepartmentDescription is weighted encoded based on the scan count of the products.  We keep the aditional column for the NaN values. All transformations are applied to the training and testing datasets...
1. We filled the Nan values with 0, only in the training dataset. 
1. We didn't drop the Upc and FinelineNumber to simplify the process, and we didn't use one hot encoding for that columns, the model went better when we just used those columns as a number. Median value of Upc per department category is used.
1. We use Random forest model.

In [28]:
def transform_data(train_data_fname, test_data_fname):
    df_train = pd.read_csv(train_data_fname)
    df_train['is_train_set'] = 1
    df_test = pd.read_csv(test_data_fname)
    df_test['is_train_set'] = 0

    # we  get the TripType for the train set. To do that, we group by VisitNumber and
    # then we get the max (or min or avg)
    y = df_train.groupby(["VisitNumber", "Weekday"], as_index=False).max().TripType
    
    # we remove the TripType now, and concat training and testing data
    # the concat is done so that we have the same columns for both datasets
    # after one-hot encoding
    df_train = df_train.drop("TripType", axis=1)
    df = pd.concat([df_train, df_test])
    
    # we drop Upc to add a median value later
    df.fillna(0, inplace=True)
    df_aux = df.drop(["ScanCount", "DepartmentDescription", "is_train_set"], axis=1)
    extra = df_aux.groupby(["VisitNumber", "Weekday"], as_index=False).median()
    df = df.drop(["Upc"], axis=1)


    # one-hot encoding for the DepartmentDescription    
    df = pd.get_dummies(df, columns=["DepartmentDescription"], dummy_na=True)
    
    # we weight the department categories based on the scan count
    departments = list(df.columns)
    departments.remove("Weekday")
    departments.remove("ScanCount")
    departments.remove("FinelineNumber")
    departments.remove("VisitNumber")
    departments.remove("is_train_set")
    #print (departments)
    df_g = df.copy()
    for dep in departments:
        df_g[dep] = df_g[dep] * df_g.ScanCount
    
    # now we add the groupby values
    df = df.groupby(["VisitNumber", "Weekday"], as_index=False).sum()
    df["Upc_median"] = extra["Upc"]
    
    # finally, we do one-hot encoding for the Weekday
    df = pd.get_dummies(df, columns=["Weekday"], dummy_na=True)
    # get train and test back
    df_train = df[df.is_train_set != 0]
    df_test = df[df.is_train_set == 0]
    
    X = df_train.drop(["is_train_set"], axis=1)
    yy = None
    XX = df_test.drop(["is_train_set"], axis=1)

    return X, y, XX, yy

In [51]:
X, y, XX, yy = transform_data("../data/train.csv", "../data/test.csv")

In [52]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [53]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

tree_param = {'criterion':('gini', 'entropy'), 'min_samples_leaf':(1),# 2, 5),
              'min_samples_split':(10)} #(2, 3, 5, 10, 50, 100)}
tree = RandomForestClassifier(random_state=42)
my_model = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)
#tree_clf = GridSearchCV(tree, tree_param, cv=3, scoring='accuracy') #scoring='balanced_accuracy')
my_model.fit(X_train, y_train)
# best_tree_clf = my_model.best_estimator_

RandomForestClassifier(min_samples_split=10, random_state=42)

In [54]:
from sklearn.metrics import mean_absolute_error

predictions = my_model.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

Mean Absolute Error: 16.432343726689542


In [55]:
from sklearn.metrics import accuracy_score

In [56]:
print("Accuracy: " + str(accuracy_score(y_valid,predictions)))

Accuracy: 0.704759063106072


**And finally**, we predict the unknown label for the testing set

In [57]:
X.shape, XX.shape

((67029, 82), (28645, 82))

In [58]:
# results dataframe is used to store the computed results
results = pd.DataFrame(columns=('clf', 'best_acc'))

In [59]:
results = results.append({'clf': predictions.reshape(-1,1), 'best_acc': accuracy_score(y_valid,predictions)}, ignore_index=True)

In [60]:
results

,clf,best_acc
0,"[[999], [7], [7], [25], [8], [7], [8], [38], [...",0.704759


In [61]:
yy = my_model.predict(XX)

The last thing we do is generating a file that should be *submitted* on kaggle

In [62]:
submission = pd.DataFrame(list(zip(XX.VisitNumber, yy)), columns=["VisitNumber", "TripType"])

In [63]:
submission

,VisitNumber,TripType
0,7,9
1,8,39
2,15,21
3,19,25
4,23,9
...,...,...
28640,191331,9
28641,191335,32
28642,191342,8
28643,191345,39


In [64]:
submission.to_csv("../data/submission.csv", header=True, index=False)